# PyTorch Modules

### What is this notebook about?

In this notebook, we will learning about PyTorch modules and the great functionalities they provide. Later on, we'll create a small a multilayer perceptron to perform image classification on MNIST.

___

## Google Colab only!

In [ ]:
# execute only if you're using Google Colab
!wget -q https://raw.githubusercontent.com/ahug/amld-pytorch-workshop/master/binder/requirements.txt -O requirements.txt
!pip install -qr requirements.txt

____

In [1]:
import torch
import torch.nn as nn

print("Torch version:", torch.__version__)

Torch version: 1.0.0


In [2]:
import matplotlib.pyplot as plt

In PyTorch, there are many predefined layer like Convolutions, RNN, Pooling, Linear, etc.

These functions are wrapped in **modules** and inherit from the **torch.nn.Module** base class.

When designing a custom model in PyTorch, you should follow this strategy and derive your class from **torch.nn.Module**.

## Modules

In [3]:
print(torch.nn.Module.__doc__)

Base class for all neural network modules.

    Your models should also subclass this class.

    Modules can also contain other Modules, allowing to nest them in
    a tree structure. You can assign the submodules as regular attributes::

        import torch.nn as nn
        import torch.nn.functional as F

        class Model(nn.Module):
            def __init__(self):
                super(Model, self).__init__()
                self.conv1 = nn.Conv2d(1, 20, 5)
                self.conv2 = nn.Conv2d(20, 20, 5)

            def forward(self, x):
               x = F.relu(self.conv1(x))
               return F.relu(self.conv2(x))

    Submodules assigned in this way will be registered, and will have their
    parameters converted too when you call :meth:`to`, etc.
    


### Modules are doing a lot of "magic" under the hood.

- It registers all the parameters of your model.
- It simplifies the saving/loading of your model.
- It provides helper functions to reset/freeze/update the gradients.
- It provides helper functions to put all parameters on a device (GPU).

### What is a torch.nn.Parameter?

A Parameter is a Tensor with `requires_grad` to `True` by default, and which is automatically added to the list of parameters when used within a model.

Let's have a look at the documentation ([torch.nn.Paramter](https://pytorch.org/docs/stable/_modules/torch/nn/parameter.html))

In [4]:
print(torch.nn.Parameter.__doc__)

A kind of Tensor that is to be considered a module parameter.

    Parameters are :class:`~torch.Tensor` subclasses, that have a
    very special property when used with :class:`Module` s - when they're
    assigned as Module attributes they are automatically added to the list of
    its parameters, and will appear e.g. in :meth:`~Module.parameters` iterator.
    Assigning a Tensor doesn't have such effect. This is because one might
    want to cache some temporary state, like last hidden state of the RNN, in
    the model. If there was no such class as :class:`Parameter`, these
    temporaries would get registered too.

    Arguments:
        data (Tensor): parameter tensor.
        requires_grad (bool, optional): if the parameter requires gradient. See
            :ref:`excluding-subgraphs` for more details. Default: `True`
    


In [5]:
mod = nn.Conv1d(10, 2, 3)
print(mod.weight)

Parameter containing:
tensor([[[ 0.0547, -0.1448, -0.0659],
         [ 0.1394,  0.1479, -0.0458],
         [-0.1138,  0.0881,  0.0599],
         [-0.1270,  0.1606, -0.1723],
         [ 0.0661, -0.1380,  0.0812],
         [-0.0090, -0.1791,  0.0325],
         [-0.1749,  0.1292,  0.1651],
         [ 0.1356, -0.1422, -0.1466],
         [-0.0909,  0.1102,  0.0068],
         [-0.1143, -0.1036,  0.0597]],

        [[ 0.0441,  0.1143, -0.0681],
         [-0.0986, -0.0605,  0.0510],
         [ 0.1437, -0.0923,  0.1040],
         [-0.1223, -0.1692, -0.1631],
         [-0.1517, -0.1361, -0.0580],
         [-0.1743, -0.1006,  0.1449],
         [-0.0101, -0.0899,  0.0027],
         [-0.0994, -0.1554,  0.0050],
         [ 0.0465, -0.0119,  0.0408],
         [ 0.1511,  0.0666, -0.1711]]], requires_grad=True)


___

## Very simple example of a module

A module has to implement two functions:

- the `__init__` function, where you define all the layers that have learnable parameters. In the `__init__` function, you are just specifying each layer and not how it is connected to others, so it does not need to be in order of execution. Since your model's submodules and parameters are instantiated in the `__init__` function, PyTorch knows that they exist and registers them.  
Also, don't forget to always call the `super()` method.  


- the `forward` function, which is the method that defines what has to be executed during the forward pass and especially how the layers are connected. This is where you call the layers that you defined inside the `__init__` function.


In [6]:
# A simple module
class MySuperSimpleModule(nn.Module):
    def __init__(self, input_size, num_classes):
        super(MySuperSimpleModule, self).__init__()  # Mandatory call to super
        self.linear = nn.Linear(input_size, num_classes)  # Define one Linear layer
    
    def forward(self, x):
        out = self.linear(x)
        return out

You can use the print function to list a model's submodules and parameters defined inside `init`:

In [7]:
model = MySuperSimpleModule(input_size=20, num_classes=5)
print(model)

MySuperSimpleModule(
  (linear): Linear(in_features=20, out_features=5, bias=True)
)


You can use **`model.parameters()`** to get the list of parameters of your model automatically inferred by PyTorch.

In [8]:
for name, p in model.named_parameters():  # Here we use a sligtly different version of the parameters() function
    print(name, ":\n", p)                 # which also returns the parameter name

linear.weight :
 Parameter containing:
tensor([[ 3.1016e-02, -1.2933e-02,  1.6506e-01,  8.7599e-02,  1.7379e-02,
          8.9231e-03,  1.6362e-01,  2.1495e-01, -1.9190e-01, -2.0273e-01,
         -1.7076e-02,  1.7849e-01,  3.3221e-02, -1.2015e-01,  1.0302e-03,
          1.9525e-01,  1.1629e-01, -5.7983e-02,  4.9398e-02, -2.1893e-01],
        [ 6.2256e-02, -1.6826e-01,  1.5629e-01, -5.2331e-02, -1.9801e-01,
          2.5363e-02, -1.4771e-01, -7.7473e-02, -2.2342e-01,  2.3809e-02,
          1.6011e-01,  2.2325e-01,  1.2546e-01,  1.6352e-01,  1.3975e-01,
         -7.9384e-02, -8.6551e-02,  4.2915e-02, -6.4083e-03,  2.0546e-01],
        [ 2.1653e-01,  1.3184e-01,  5.2778e-02, -1.7163e-01, -5.9402e-02,
         -1.1593e-01,  1.2754e-02, -5.2815e-02, -1.9686e-01, -4.8503e-02,
          4.7428e-02,  1.0217e-01, -4.6087e-02,  8.7236e-02, -1.3228e-01,
          1.2760e-01, -1.1586e-01,  1.8095e-01, -9.8876e-02, -1.6028e-01],
        [ 1.2663e-01,  1.7606e-01, -1.4559e-01,  7.7750e-02, -1.7729e-

___

## Simple network for image classification

![We need to go depper](figures/deeper.jpeg)

## Your turn!

### Let's create a more complicated model.

Implement a simple multilayer perceptron with two hidden layers and the following structure:

![](https://raw.githubusercontent.com/ledell/sldm4-h2o/master/mlp_network.png)

- Input-size: *input_size*
- 1st hidden layer: 75
- 2nd hidden layer: 50
- Output layer: *num_classes*

Additionally, we use `ReLU`s as activation functions.

You will need some PyTorch NN modules - Find them in the [PyTorch doc](https://pytorch.org/docs/master/nn.html) (especially nn.Linear)!

In [ ]:
from torch.nn import Parameter
import torch.nn.functional as F  # provides some helper functions like Relu's, Sigmoids, Tanh, etc.


class MyMultilayerPerceptron(nn.Module):
    def __init__(self, input_size, num_classes):
        super(MyMultilayerPerceptron, self).__init__()
        
        self.input_size = input_size
        self.num_classes = num_classes
        
        self.linear_1 = nn.Linear(input_size, 75)
        self.linear_2 = # <YOUR CODE>
        self.linear_3 = nn.Linear(150, num_classes)
        
    
    def forward(self, x):
        out = F.relu(self.linear_1(x))
        out = # <YOUR CODE>
        out = # <YOUR CODE>
        return out

### Print your network's parameters

In [ ]:
model = MyMultilayerPerceptron(784, 10)
print(model)

### Feed an input to your network

In [ ]:
x = torch.rand(16, 784)  # the first dimension is reserved for the 'batch_size'
out = model(x)  # equivalent to model.forward(x)
out[0, :]

___

## Training a model

Most of the functions to train a model follow a similar pattern in PyTorch.
In most of the cases in consists of the following steps:
- Loop over data (in batches)
- Forward pass
- Zero gradients!
- Backward pass
- Parameter update (Optimizer)

In [ ]:
def train(model, num_epochs, data_loader, device):
    model = model.to(device)
    
    # Define the Loss function and Optimizer that you want to use
    criterion = nn.CrossEntropyLoss()  
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)  # NOTE: model.parameters()
    
    # Outter training loop
    for epoch in range(num_epochs):
        # Inner training loop
        cum_loss = 0
        for (inputs, labels) in train_loader:
            # Prepare inputs and labels for processing by the model (e.g. reshape, move to device, ...)
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            # original shape is [batch_size, 28, 28] because it's an image of size 28x28
            inputs = inputs.view(-1, 28*28)

            # Do Forward -> Loss Computation -> Backward -> Optimization
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            cum_loss += loss.item()
        print("Epoch %d, Loss=%.4f" % (epoch+1, cum_loss/len(train_loader)))

Note:
- we can use the `.to` function on the model directly. Indeed, since PyTorch knows all the model parameters, it can put all the parameters on the correct device.
- we use `model.parameters()` to get all the parameters of the model and we can instantiate an optimizer that will optimize these parameters `torch.optim.SGD(model.parameters())`.
- to apply the forward function of the module, we write `model(input)`. In most cases, `model.forward(inputs)` would also work, but there is a slight difference : PyTorch allows you to register hook functions for a model that are automatically called when you do a forward pass on your model. Using `model(input)` will call these hooks and then call the forward function, while using `model.forward(inputs)` will just silently ignore them.

Do you feel the power of Modules ?

## Loss functions

PyTorch comes with a lot of predefined loss functions :
- L1Loss
- MSELoss
- CrossEntropyLoss
- NLLLoss
- PoissonNLLLoss
- KLDivLoss
- BCELoss
- MarginRankingLoss
- HingeEmbeddingLoss
- MultiLabelMarginLoss
- CosineEmbeddingLoss
- TripletMarginLoss
- ...

Check out the [PyTorch Documentation](https://pytorch.org/docs/master/nn.html#loss-functions).

___

## Let's train our model on the MNIST digit classification task


![MNIST](figures/mnist.jpeg)

First, we have to load the training and test images. MNIST is a widely used dataset, therefore the torchvision package provides simple functionalities to load images from it.

In [ ]:
import torchvision.datasets as datasets
import torchvision.transforms as transforms

batch_size = 64

# MNIST Dataset (Images and Labels)
train_dataset = datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor())

# Dataset Loader (Input Batcher)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In PyTorch, `Dataset` and `Dataloaders` are classes that can help to quickly define how to access and iterate over your data. This is specially interesting when your data is distributed over several files (for instance, if you have several images in some directory structure).

### Call the actual training function

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MyMultilayerPerceptron(input_size=784, num_classes=10)
num_epochs = 5

train(model, num_epochs, train_loader, device)

### How can we now assess the model's performance?

This function loops over another `data_loader` (usually containing test/validation data) and computes the model's accuracy on it.

In [ ]:
def accuracy(model, data_loader, device):
    with torch.no_grad(): # during model evaluation, we don't need the autograd mechanism (speeds things up)
        correct = 0
        total = 0
        for inputs, labels in data_loader:
            inputs = inputs.to(device)     
            inputs = inputs.view(-1, 28*28)
            
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            
            correct += (predicted.cpu() == labels).sum().item()
            total += labels.size(0)
            
    acc = correct / total
    return acc

In [ ]:
accuracy(model, test_loader, device)  # look at: accuracy(model, train_loader, device)

### We get an accuracy of ~97.9%, can we do better?

____

## How can we now store our trained model?

In [ ]:
torch.save(model, "my_model.pt")

In [ ]:
my_model_loaded = torch.load("my_model.pt")

In [ ]:
model.linear_3.bias, my_model_loaded.linear_3.bias

____

This intro to modules used [this medium post](https://medium.com/deeplearningbrasilia/deep-learning-introduction-to-pytorch-5bd39421c84) as a resource.

## Don't forget to download the notebook, otherwise your changes may be lost!

![Download the notebook](figures/notebook-download.png)